In [1]:
import sys 
sys.path.append("..") 
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils import data_helper,result_helper
from models import lstm_model

In [2]:
# 读取关键词命中结果数据

train_ds7='../data/train/unlabeled_data_labeled7_v1.csv'
train_ds7_plus='../data/train/unlabeled_data_labeled7_plus.csv'
train_ds3='../data/train/unlabeled_data_labeled3_v1.csv'

train_df7=pd.read_csv(train_ds7)
train_df7_plus=pd.read_csv(train_ds7_plus)
train_df3=pd.read_csv(train_ds3)

print(train_df7['class_label'].value_counts())
print(train_df7_plus['class_label'].value_counts())
print(train_df3['class_label'].value_counts())

时尚    2337
财经    1939
房产    1914
科技    1220
家居     829
时政     738
教育     432
Name: class_label, dtype: int64
教育    1753
家居     452
时政     429
Name: class_label, dtype: int64
娱乐    4207
游戏    3061
体育    1070
Name: class_label, dtype: int64


In [3]:
# 读取已标注数据
train_ds1='../data/train/labeled_data_processed.csv'
train_df1=pd.read_csv(train_ds1)

In [4]:
class_label_to_class_dict = {
            0: '家居', 1: '房产', 2: '教育', 3: '时尚', 4: '时政', 5: '科技', 6: '财经', 7: '游戏', 8: '娱乐', 9: '体育'}

class_to_class_label_dict={val:key for key,val in class_label_to_class_dict.items()}
class_to_class_label_dict

{'家居': 0,
 '房产': 1,
 '教育': 2,
 '时尚': 3,
 '时政': 4,
 '科技': 5,
 '财经': 6,
 '游戏': 7,
 '娱乐': 8,
 '体育': 9}

In [5]:
# 合并关键词命中数据
train_df2=train_df7[['content','cutted_content','class_label','by_words']].append(train_df7_plus[['content','cutted_content','class_label','by_words']])
train_df2=train_df2.append(train_df3[['content','cutted_content','class_label','by_words']])

# 添加标签列
train_df2['label']=train_df2['class_label'].apply(lambda x:class_to_class_label_dict[x])

# 合并已标注数据和关键词命中数据
train_df=train_df1[['content','cutted_content','class_label','label']].append(train_df2)
print(train_df['class_label'].value_counts())
train_df=train_df.drop_duplicates(subset=['content']) # 去除重复的content
print(train_df['class_label'].value_counts())


娱乐    4207
时尚    3337
教育    3185
游戏    3061
财经    2939
房产    2914
家居    2281
科技    2220
时政    2167
体育    1070
Name: class_label, dtype: int64
娱乐    3610
时尚    3336
游戏    2940
财经    2925
房产    2910
教育    2853
科技    2220
家居    1888
时政    1800
体育     993
Name: class_label, dtype: int64


In [6]:
save_k=int(np.mean(train_df['class_label'].value_counts()))

# 后三类每个保留最多k条数据
train_df_sub=pd.DataFrame()
for label in train_df['label'].unique():
    train_df_sub=train_df_sub.append(train_df[train_df['label']==label].iloc[:save_k])
train_df_sub

In [7]:
# 打乱数据
train_df=train_df.sample(frac=1).reset_index(drop=True)
train_df

,content,cutted_content,class_label,label,by_words
0,群侠传中对于侠客们来说最重要的一个就是这个天赋上的选择了，带有正确天赋的侠客对于玩家们来说可...,群侠传 中 侠客 来说 最 一个 天赋 上 选择 带有 正确 天赋 侠客 玩家 来说 说 强...,游戏,7,礼包 副本
1,NCAA主席马克·埃默特(MarkEmmert)和董事会周四在一份声明中表示，NCAA正式取...,NCAA 主席 马克 埃默特 MarkEmmert 董事会 周四 一份 声明 中 NCAA ...,体育,9,NBA 锦标赛
2,合景领汇长安是西长安街上双地铁纯商品房住区，合景领汇长安在哪里?项目位于门头沟新城繁华地段。...,合景 领汇 长安 西长安街 上 双 地铁 纯 商品房 住区 合景 领汇 长安 项目 位于 门...,房产,1,公园 建筑面积
3,据周五(11月8日)FX168每周金融市场调查显示，在WTI原油期货本周温和上升后，分析师和...,周五 11 月 8 日 FX168 每周 金融市场 调查 显示 WTI 原油期货 本周 温和...,财经,6,NaN
4,北京时间9月2日，弗兰西斯科-莫里纳利(FrancescoMolinari)已经放弃了年度第...,北京 时间 9 月 2 日 弗兰西斯 科莫 里纳 利 FrancescoMolinari 放...,体育,9,奥运会 锦标赛
...,...,...,...,...,...
25470,中新网客户端8月27日电(郎朗)27日，教育部召开新闻发布会。会上介绍，今年秋季学期处于常态...,中新网 客户端 8 月 27 日电 郎朗 27 日 教育部 新闻 发布会 会上 介绍 秋季 ...,教育,2,NaN
25471,尼尔森25日发布报告指出，近段时间以来中国零售业持续复苏，呈现出“U”型走势。据悉，中国是全...,尼尔森 25 日 发布 报告 指出 近 段时间 中国 零售业 持续 复苏 呈现出 U 型 走...,家居,0,全屋 家居
25472,由武汉玛雅动漫、鑫岳影视联合出品，上海鑫岳独家宣发，曾宪林自编自导的国内首部“趣”科幻、“软...,武汉 玛雅 动漫 鑫岳 影视 联合 出品 上海 鑫岳 独家 宣发 曾宪林 自编 自导 国内 ...,娱乐,8,传媒 影片
25473,公开招标竞价优选精准采购 杭州余杭区用大数据守护学生盘中餐 本报讯（记者蒋亦丰通讯员武怡...,公开招标 竞价 优选 精准 采购 杭州 余杭区 用大 数据 守护 学生 盘中餐 本报讯 记者...,教育,2,NaN


In [8]:
train_df.to_csv('../data/train/train_processed_v2.csv',index=False)